<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/langchain_fulltext_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install neo4j openai langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

graph = Neo4jGraph(
    url="bolt://3.238.222.252:7687",
    username="neo4j",
    password="bushels-harpoon-deduction"
)

In [13]:
import os

os.environ['OPENAI_API_KEY'] = "sk-"

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0, model_name="gpt-4-0613"), graph=graph, verbose=True,
    return_direct=True
)

In [14]:
chain.run("Who played in the Matrix?")



> Entering new  chain...
Generated Cypher:
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) WHERE m.title = 'Matrix' RETURN a.name

> Finished chain.


[]

In [15]:
graph.query("""CREATE FULLTEXT INDEX entities IF NOT EXISTS
FOR (n:Movie|Person)
ON EACH [n.name, n.title]
""")

[]

In [16]:
graph.query("""
CALL db.index.fulltext.queryNodes("entities", "Matrix") YIELD node, score
RETURN node.title, score
LIMIT 3""")

[{'node.title': 'Matrix, The', 'score': 4.259097099304199},
 {'node.title': 'Matrix Revolutions, The', 'score': 3.7098255157470703},
 {'node.title': 'Matrix Reloaded, The', 'score': 3.7098255157470703}]

In [17]:
from __future__ import annotations
from langchain.chains.base import Chain

from typing import Any, Dict, List

from pydantic import Field


fulltext_search = """
CALL db.index.fulltext.queryNodes("entities", $query)
YIELD node, score
RETURN coalesce(node.title, node.name) AS option, labels(node)[0] AS type LIMIT 3
"""


class Neo4jFulltextGraphChain(Chain):
    """Chain for keyword question-answering against a graph."""

    graph: Neo4jGraph = Field(exclude=True)
    input_key: str = "query"  #: :meta private:
    output_key: str = "result"  #: :meta private:

    @property
    def input_keys(self) -> List[str]:
        """Return the input keys.
        :meta private:
        """
        return [self.input_key]

    @property
    def output_keys(self) -> List[str]:
        """Return the output keys.
        :meta private:
        """
        _output_keys = [self.output_key]
        return _output_keys

    def _call(self, inputs: Dict[str, str]) -> Dict[str, Any]:
        """Extract entities, look up info and answer question."""
        question = inputs[self.input_key]
        context = self.graph.query(
            fulltext_search, {'query': question})
        return {self.output_key: context}


In [18]:
fulltext = Neo4jFulltextGraphChain(graph=graph)
fulltext.run("Matrix")

[{'option': 'Matrix, The', 'type': 'Movie'},
 {'option': 'Matrix Revolutions, The', 'type': 'Movie'},
 {'option': 'Matrix Reloaded, The', 'type': 'Movie'}]

In [19]:
fulltext.run("Keanu")

[{'option': 'Keanu', 'type': 'Movie'},
 {'option': 'Keanu Reeves', 'type': 'Actor'}]

In [20]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
tools = [
    Tool(
        name="Movies",
        func=chain.run,
        description="""useful for when you need to information about movies, actors, directors, and so on.
        Input must be full question.
        Always make sure to use the entity search first to validate the names of movies and person.""",
    ),
    Tool(
        name="Entity search",
        func=fulltext.run,
        description="""useful for when you need find exact values of names of people and movies. The tool returns three options and you have to select the best one.
        Input must be a single entity.""",
    ),

]

In [21]:
graph_agent = initialize_agent(tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [22]:
graph_agent.run("Who played in the Matrix?")



> Entering new  chain...
Thought: I should use the Movies tool to get information about the movie "The Matrix" and its cast.
Action:
```
{
  "action": "Movies",
  "action_input": "cast of The Matrix"
}
```


> Entering new  chain...
Generated Cypher:
MATCH (m:Movie {title: 'The Matrix'})-[:ACTED_IN]-(a:Actor)
RETURN a.name

> Finished chain.

Observation: []
Thought:The observation should not be an empty list. Let me try again.
Action:
```
{
  "action": "Movies",
  "action_input": "The Matrix"
}
```


> Entering new  chain...
Generated Cypher:
MATCH (m:Movie) WHERE m.title = 'The Matrix' RETURN m

> Finished chain.

Observation: []
Thought:I need to check the exact name of the movie using the Entity search tool first.
Action:
```
{
  "action": "Entity search",
  "action_input": "The Matrix"
}
```


Observation: [{'option': 'Matrix, The', 'type': 'Movie'}, {'option': 'Matrix Reloaded, The', 'type': 'Movie'}, {'option': 'Matrix Revolutions, The', 'type': 'Movie'}]
Thought:I can now use

OutputParserException: ignored